# Import libraries

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
from joblib import dump

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from  sklearn.pipeline  import  Pipeline
from  sklearn  import  set_config
set_config(display='diagram')

In [6]:
from lightgbm import LGBMClassifier

# Custom functions

In [7]:
def dataset_parameters(df,target):
    df_features = df.drop([target], axis=1)
    data_type_numerical = ['int64', 'float64']
    data_type_object = ['object', 'bool', 'category']
    data_type_date = ['datetime64']
    categorical_features  = list(filter(lambda x: df_features[x].dtype in data_type_object, df_features.columns))
    numerical_features = list(filter(lambda x: df_features[x].dtype in data_type_numerical, df_features.columns))
    date_features = list(filter(lambda x: df_features[x].dtype in data_type_date, df_features.columns))
    all_features  = numerical_features  + categorical_features + date_features 
    return categorical_features, numerical_features, date_features, all_features,target 

# Loading and set data

In [8]:
with open('airline_df.pkl', 'rb') as f:
    df = pickle.load(f)

In [9]:
df

,Gender,customer_type,age,type_of_travel,customer_class,flight_distance,inflight_wifi_service,departure_arrival_time_convenient,ease_of_online_booking,gate_location,...,inflight_entertainment,onboard_service,leg_room_service,baggage_handling,checkin_service,inflight_service,cleanliness,departure_delay_in_minutes,arrival_delay_in_minutes,satisfaction
0,Male,Loyal Customer,13,Personal Travel,Eco Plus,460,3,4,3,1,...,5,4,3,4,4,5,5,25,18.0,neutral or dissatisfied
1,Male,disloyal Customer,25,Business travel,Business,235,3,2,3,3,...,1,1,5,3,1,4,1,1,6.0,neutral or dissatisfied
2,Female,Loyal Customer,26,Business travel,Business,1142,2,2,2,2,...,5,4,3,4,4,4,5,0,0.0,satisfied
3,Female,Loyal Customer,25,Business travel,Business,562,2,5,5,5,...,2,2,5,3,1,4,2,11,9.0,neutral or dissatisfied
4,Male,Loyal Customer,61,Business travel,Business,214,3,3,3,3,...,3,3,4,4,3,3,3,0,0.0,satisfied
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129482,Male,disloyal Customer,34,Business travel,Business,526,3,3,3,1,...,4,3,2,4,4,5,4,0,0.0,neutral or dissatisfied
129483,Male,Loyal Customer,23,Business travel,Business,646,4,4,4,4,...,4,4,5,5,5,5,4,0,0.0,satisfied
129484,Female,Loyal Customer,17,Personal Travel,Eco,828,2,5,1,5,...,2,4,3,4,5,4,2,0,0.0,neutral or dissatisfied
129485,Male,Loyal Customer,14,Business travel,Business,1127,3,3,3,3,...,4,3,2,5,4,5,4,0,0.0,satisfied


In [10]:
categorical_features, numerical_features, date_features, all_features, target = dataset_parameters(df,'satisfaction')

In [11]:
X = df[all_features]

In [12]:
y = df[target]

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=13) 

# Setup pipeline, train and test with best hyperparameters

In [14]:
categorical_transformer = Pipeline(steps=[
    ('one' , OneHotEncoder())
])

In [15]:
numerical_transformer = Pipeline(steps=[
    ('scale', StandardScaler()),
])

In [16]:
preprocessor = ColumnTransformer(transformers=[
    ('num', numerical_transformer, numerical_features),
    ('cat', categorical_transformer, categorical_features)
])

In [17]:
params = {
     'boosting_type': 'dart',
     'n_estimators': 729,
     'min_child_samples': 5,
     'learning_rate': 0.09058025591953338,
     'num_leaves': 146,
     'colsample_bytree': 0.9773512265322472,
     'reg_alpha': 0.04836405507163106,
     'reg_lambda': 0.08467358653637265,
     'min_split_gain': 0.6060617236332217,
     'min_child_weight': 0.19214701592888236
}

In [18]:
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', LGBMClassifier(objective="binary", **params))
])

In [19]:
pipeline.fit(X_train,  y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('scale',
                                                                   StandardScaler())]),
                                                  ['age', 'flight_distance',
                                                   'inflight_wifi_service',
                                                   'departure_arrival_time_convenient',
                                                   'ease_of_online_booking',
                                                   'gate_location',
                                                   'food_and_drink',
                                                   'online_boarding',
                                                   'seat_comfort',
                                                   'inflight_entertainment',
                                                   'onboard_service',
                                                   'leg_room_servi...
                                                   'type_of_travel',
                                                   'customer_class'])])),
                ('model',
                 LGBMClassifier(boosting_type='dart',
                                colsample_bytree=0.9773512265322472,
                                learning_rate=0.09058025591953338,
                                min_child_samples=5,
                                min_child_weight=0.19214701592888236,
                                min_split_gain=0.6060617236332217,
                                n_estimators=729, num_leaves=146,
                                objective='binary',
                                reg_alpha=0.04836405507163106,
                                reg_lambda=0.08467358653637265))])

In [20]:
pipeline.score(X_test,y_test)

0.9664581563569903

# Save pipeline to disk

In [21]:
dump(pipeline, 'pipeline.joblib')

['pipeline.joblib']